In [524]:
from functools import reduce
import statistics as st
import cv2
import pytesseract
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from datetime import datetime
#import matplotlib.pyplot as plt
import os
#from PyPDF2 import PdfReader
#%matplotlib inline
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\epenago\AppData\Local\Tesseract-OCR\tesseract.exe'

In [525]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

In [526]:
def calcular_espacios(diccionario, indices):
    
    suma = 0
    bandera = True
  
    width_espacios = []
    for i in range(len(indices)):
             
        if(bandera == True):
            
            bandera = False
            suma = 0
            suma += int(diccionario["left"][indices[i]])+int(diccionario["width"][indices[i]])

            if i < len(indices)-1:
                if int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+10 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-10:
                    pass
                else:
                    bandera = True
            else:           
                    bandera = True
                    

        elif(i < len(indices)-1):
            width_espacios.append(diccionario["left"][indices[i]] - suma)
            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            if int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+10 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-10:
                pass
            else:
                bandera = True
               

        elif(i <= len(indices)-1):
            width_espacios.append(diccionario["left"][indices[i]] - suma)
            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            bandera = True
            
    #print(width_espacios)
    width_espacios.sort() 
    
    moda = st.mode(width_espacios)
    return (moda-(moda*1), moda+(moda*1))

In [527]:
def coordenadas_extraccion(diccionario, indices, intervalo):
    
    datos_extraidos = []
    dict = {}
    coordenadas = []
    suma = 0
    bandera = True
    indice_anterior = -1
    strings = []
    string = ""
    index = []
    line = 0

    for i in range(len(indices)):
             
        if(bandera == True):
            indice_anterior = i
            
            bandera = False
            suma = 0
            suma += int(diccionario["left"][indices[i]])+int(diccionario["width"][indices[i]])

            # Se guarda la palabra y su indice
            string = diccionario["text"][indices[i]]
            index.append(indices[i])
            
            if i < len(indices)-1:
                if (int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+13 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-13):
                    
                    if (diccionario["left"][indices[i+1]] - suma) < intervalo[0] or (diccionario["left"][indices[i+1]] - suma) > intervalo[1]:
                        bandera = True
                        #print(indices[i])
                        coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                        diccionario["top"][indices[indice_anterior]], 
                        suma,
                        diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])
                        
                        dict["text"] = string
                        dict["line"] = line
                        dict["index"] = index
                        dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                        datos_extraidos.append(dict)
                                                                    
                        dict = {}                        
                        string = ""
                        index = []

                        bandera = True
                        indice_anterior = -1
                else:
                    bandera = True
                    #print(indices[i])
                    coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                    diccionario["top"][indices[indice_anterior]], 
                    suma,
                    diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])
                    

                    dict["text"] = string
                    dict["line"] = line
                    dict["index"] = index
                    dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                    datos_extraidos.append(dict)
                                                                
                    dict = {}                        
                    string = ""
                    index = []
                    line += 1

                    bandera = True
                    indice_anterior = -1
            else:           
                    coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                    diccionario["top"][indices[indice_anterior]], 
                    suma,
                    diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

                    dict["text"] = string
                    dict["line"] = line
                    dict["index"] = index
                    dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                    datos_extraidos.append(dict)
                                                                
                    dict = {}                        
                    string = ""
                    index = []

                    bandera = True
                    indice_anterior = -1

        elif(i < len(indices)-1):
            string += (" " + diccionario["text"][indices[i]])
            index.append(indices[i])

            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            if (int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+13 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-13):
                
                if (diccionario["left"][indices[i+1]] - suma) < intervalo[0] or (diccionario["left"][indices[i+1]] - suma) > intervalo[1]:
                    bandera = True
                    #print(indices[i])
                    coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                    diccionario["top"][indices[indice_anterior]], 
                    suma,
                    diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

                    dict["text"] = string
                    dict["line"] = line
                    dict["index"] = index
                    dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                    datos_extraidos.append(dict)
                                                                
                    dict = {}                        
                    string = ""
                    index = []

                    bandera = True
                    indice_anterior = -1
            else:
                bandera = True
                #print(indices[i])
                coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                diccionario["top"][indices[indice_anterior]], 
                suma,
                diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

                dict["text"] = string
                dict["line"] = line
                dict["index"] = index
                dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                datos_extraidos.append(dict)
                                                            
                dict = {}                        
                string = ""
                index = []
                line += 1

                bandera = True
                indice_anterior = -1

        elif(i <= len(indices)-1):
            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            coordenadas.append([diccionario["left"][indices[indice_anterior]], 
            diccionario["top"][indices[indice_anterior]], 
            suma,
            diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

            string = ""
            index = []

            bandera = True
            indice_anterior = -1

    return (coordenadas, datos_extraidos)

En el Metodo extraccion_columns logramos separar las columnas extrayedo y clasificando los indices en el dicionario data_ordenaizer

In [528]:
def extracion_colums(df):

    arr = np.array(df["coordinates"].to_list())
    colums = []
    
    for i in np.arange(len(arr)):
        temp = []
        temp.append(i)
        for j in np.arange(len(arr)):

            if i == j: continue
            interval1 = np.arange(arr[i,0], arr[i,1]+1, dtype=int)
            interval2 = np.arange(arr[j,0], arr[j,1]+1, dtype=int)

            sum = 0
            for k in interval1:
                if k in interval2:
                    sum += 1
    
            tamaño1 = len(interval1)
            tamaño2 = len(interval2)
            percent1 = int((100/tamaño1)*sum )
            percent2 = int((100/tamaño2)*sum )
            """
            print("tamaño1: ", tamaño1)
            print("tamaño2: ", tamaño2)
            print("la suma: ", sum)
            print("percent1: ", percent1, "%")
            print("percent2: ", percent2, "%")
            """
            if percent1 > 55 and percent2 > 55:
                temp.append(j)
        
        colums.append(temp)
        
    cl = []
    descartados = []
    for i in np.arange(len(colums)):
        temp = []
        temp = colums[i]
        if i in descartados: continue
        for j in np.arange(len(colums)):

            if i == j: continue
            interval1 = colums[i]
            interval2 = colums[j]

            sum = 0
            for k in interval1:
                if k in interval2:
                    sum += 1
    
            tamaño1 = len(interval1)
            tamaño2 = len(interval2)
            percent1 = int((100/tamaño1)*sum )
            percent2 = int((100/tamaño2)*sum )
            """
            print("tamaño1: ", tamaño1)
            print("tamaño2: ", tamaño2)
            print("la suma: ", sum)
            print("percent1: ", percent1, "%")
            print("percent2: ", percent2, "%")
            """
            if percent1 > 20 or percent2 > 20:
                temp += colums[j]
                descartados.append(j)
        cl.append(temp)

    colums = []    
    #se eliminan elementos repetidos en las columnas
    for element in cl:
        temp = []
        for i in element:
            if i not in temp:
                temp.append(i)
        colums.append(temp)
    

    print("el tamaño de cl es: ", len(cl))
    #en este bloque se saca la pocision en pixeles de la imagen
    min_max = []
    for element in colums:   
        menor = arr[element[0],0]
        mayor = arr[element[0],1]
        for i in element:
            if arr[i,0] < menor:
                menor = arr[i,0]
            
            if arr[i,1] > mayor:
                mayor = arr[i,1]
        min_max.append((menor,mayor))

    for i in range(len(min_max)):
        print(min_max[i], colums[i])

    cl = []
    descartados = []
    by_delete = []
    for i in range(len(min_max)):
        temp= []
        temp= colums[i]
        if i in descartados: continue
        for j in range(len(min_max)):

            if i==j: continue
            if min_max[i][0]-10 <= min_max[j][0] and min_max[i][1]+10 >= min_max[j][1]:
                temp += colums[j]
                descartados.append(j)
                if i > j:
                    by_delete.append(colums[j])
        cl.append(temp)

    colums = cl
    print("estos son a eliminar", by_delete)
    for k in by_delete:
        if k in colums:
            colums.remove(k)

    print()
    zises = []
    for element in colums:
        print(element)
        zises.append(len(element))
    
    mean = st.mean(zises)
    print("la media de los len ", mean)
    by_delete = []
    for i in range(len(colums)):
        percent20 = (mean/100)*20
        #print("percent20: ", percent20)
        if len(colums[i]) < percent20:
            by_delete.append(colums[i])

    for element in colums:
        print(element)
        print

    print("elementos a eliminar: ", by_delete)
    for k in by_delete:
        if k in colums:
            colums.remove(k)

    for element in colums:
        print(element)

    min_max_h= []
    for element in colums:   
        menor = arr[element[0],0]
        mayor = arr[element[0],1]
        for i in element:
            if arr[i,0] < menor:
                menor = arr[i,0]
            
            if arr[i,1] > mayor:
                mayor = arr[i,1]
        min_max_h.append((menor,mayor))

    min_max_h_unOrder = min_max_h.copy()
    min_max_h.sort()

    cl = []
    for i in np.arange(len(colums)):
        for j in np.arange(len(colums)):
            if min_max_h_unOrder[j] == min_max_h[i]:
                cl.append(colums[j])

    colums = cl

    print(min_max_h)
    
    return min_max_h,colums

El metodo build_df toma el arreglo con las columnas separadar y utiliza el diccionario construido en extracion_colums 
para crear un objeto tipo pandas.DataFrame con la misma distribucion de los datos que estaba en la imagen

In [529]:
def build_df(pd_dataOrdenizer, colums):
    temp = pd.DataFrame()
    pd_columns = pd.DataFrame()
    for i in range(len(colums)):
        temp = pd.DataFrame.copy(pd_dataOrdenizer.loc[colums[i], ["text", "line"]])
        temp = temp.drop_duplicates()
        temp = temp.sort_values("line")
        temp = pd.Series(temp["text"].tolist(), index=temp["line"].tolist())
        temp = temp.loc[~temp.index.duplicated(keep='first')]
        pd_columns = pd.concat([pd_columns, temp], axis=1, ignore_index=True)

    pd_columns = pd_columns.sort_index()
    return pd_columns

In [530]:
def del_headEnd(df):

    freq = df['line'].value_counts().sort_index().head(10)
    to_delete = []
    bandera = True

    for i in range(len(freq)):
        if bandera == True:
            if freq[i] == 1:
                to_delete.append(i)
            
            elif freq[i] == 2 and freq[i+1] == 1:
                to_delete.append(i)
            else: bandera = False
    
    for i in to_delete:
        df.drop(df.index[df['line'] == i] , axis=0, inplace=True)
    df.reset_index(inplace=True)
    df.drop(['level_0'], axis=1, inplace=True)
    

In [531]:
def isTime(string):
    try:
        date = datetime.strptime(string	, '%d/%m/%Y')
        return str(date.year)
    except ValueError:
        return np.nan


In [532]:
def isNumeric(s):
    try:
        if type(s) != float:
            s = s.replace('(','') 
            s = s.replace(')','') 
        float(s)
        return True
    except ValueError:
        return False

In [533]:
def convert_parenthesis(s):
   
    if type(s) != float:
        if '(' in s or ')' in s:
            s = s.replace('(','') 
            s = s.replace(')','')
            s = '-' + s
    
    return s


In [534]:
def isNumericColum(df): 
    type_index=[]                   
    str_index=[]
    counter_num = 1

    for i in df.columns:
        df[i] = df.apply(lambda x: x[i].replace(',','') if type(x[i]) == str else x[i], axis=1)
        df[i] = df.apply(lambda x: x[i].replace('.','') if type(x[i]) == str else x[i], axis=1)
        df[i] = df.apply(lambda x: '0' if x[i] == 'o' else x[i], axis=1)

    for i in df.columns:
        size = df[i].apply(isNumeric).value_counts().sum()
        try:
            if 100*df[i].apply(isNumeric).value_counts()[True]/size >= 80:
                type_index.append('num')
                df[i] = df[i].apply(convert_parenthesis)
            else:
                type_index.append('text')
            
        except KeyError:
            type_index.append('text')

    years = np.arange(2018,datetime.now().year + 1)
    years = [str(i) for i in years]
    years.append('NOTAS')
    for i in df.columns:
        if type_index[i] == 'text':
            str_index.append(df[i][~df[i].isnull()].head(1).to_list()[0])
            #to_delete.append(df[i][~df[i].isnull()].head(1).index.to_list()[0])
        else:
            
            df_three = df[i][~df[i].isnull()].head(3).apply(lambda x: x if x.upper() in years else np.nan)
            if df_three.count() == 0:
                df_three = df[i][~df[i].isnull()].head(3).apply(isTime)
                if df_three.count() == 0:
                    if len(df[i][~df[i].isnull()].head(3)[~df[i][~df[i].isnull()].apply(isNumeric).head(3)].to_list()) == 0:
                        str_index.append('num' + str(counter_num))
                        counter_num += 1
                        
                    else:
                        str_index.append(df[i][~df[i].isnull()].head(3)[~df[i][~df[i].isnull()].apply(isNumeric).head(3)].to_list()[0])

                else:
                    str_index.append(df_three[~df_three.isnull()].head(1).to_list()[0])
            else:
                str_index.append(df_three[~df_three.isnull()].head(1).to_list()[0])
                #to_delete.append(df[i][~df[i].isnull()].head(1).index.to_list()[0])

    str_index = [i.upper() for i in str_index]

    return str_index, type_index

In [535]:
def conver_to_float(x):
    
    
    if ',' in x and '.' in x or len(x.replace(',','.').split('.')[-1]) != 3 or len(x.replace('.',',').split('.')[-1]) != 3:

            x = x.replace(',','.')
            x = x.replace('.','',x.count('.')-1)
            num_decimales = len(x.split('.')[-1])
        
            x = float(x)
    else:
        x = x.replace(',','')
        x = x.replace('.','')
        x = float(x)

    return x

In [536]:
#main
#path = r'./reto/1.PNG'
path = r'.\reto\4.PNG'
image = cv2.imread(path)
gray = get_grayscale(image)
th = thresholding(gray)
openingg = opening(gray)
cannyy = canny(gray)
img = image
hImg, wImg, _ = img.shape

custom_config = r'-c tessedit_char_blacklist=$¢§_{} --oem 3 --psm 4' 
data = pytesseract.image_to_data(img, lang= 'eng+spa', config=custom_config, output_type= 'dict')


In [537]:
pd.DataFrame(data).tail(50)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
118,5,1,4,7,3,3,403,457,8,12,96.359680,7
119,5,1,4,7,3,4,536,457,89,12,96.858101,11.178.000
120,5,1,4,7,3,5,728,457,80,12,96.682449,6.048.000
121,5,1,4,7,3,6,908,457,80,12,79.476433,5.130.000
122,5,1,4,7,3,7,1016,458,30,11,85.709908,85%
123,4,1,4,7,4,0,33,480,1018,12,-1,
124,5,1,4,7,4,1,33,480,39,12,95.340736,Total
125,5,1,4,7,4,2,80,480,56,12,95.855049,Pasivo
126,5,1,4,7,4,3,142,480,74,12,96.864082,Corriente
127,5,1,4,7,4,4,525,480,100,12,96.770981,102.524.000


In [538]:
indice_words = []
for i in range(len(data['text'])):
    if data['text'][i] != '' and data['text'][i] != ' ' and float(data['conf'][i]) >= 50 and data["text"][i] != "-" and data["text"][i] != "--" and data["text"][i] != "s" and data["text"] != "S":
        indice_words.append(i)

#print(len(indice_words))
#print(indice_words)
intervalo = calcular_espacios(data, indice_words)
print(intervalo)
coordenadas,data_organizer = coordenadas_extraccion(data, indice_words, intervalo)
df_data_organizer = pd.DataFrame(data_organizer)

(0, 12)


In [539]:
df_data_organizer.head(10)

,text,line,index,coordinates
0,Notas,0,[8],"(385, 430)"
1,2021,0,[9],"(526, 564)"
2,2020,0,[10],"(703, 742)"
3,VARIACION,0,[11],"(862, 945)"
4,ACTIVOS,1,[20],"(33, 108)"
5,CORRIENTE,2,[24],"(103, 201)"
6,Efectivo,3,[27],"(34, 94)"
7,1,3,[28],"(403, 409)"
8,12.068.571,3,[29],"(531, 617)"
9,14.864.078,3,[30],"(714, 803)"


In [540]:
df_data_organizer.loc[:,"coordinates"].apply(lambda x: (x[0]+x[1])/2).head(10)

0    407.5
1    545.0
2    722.5
3    903.5
4     70.5
5    152.0
6     64.0
7    406.0
8    574.0
9    758.5
Name: coordinates, dtype: float64

In [541]:
del_headEnd(df_data_organizer)

In [542]:
df_data_organizer.head(5)

,text,line,index,coordinates
0,Notas,0,[8],"(385, 430)"
1,2021,0,[9],"(526, 564)"
2,2020,0,[10],"(703, 742)"
3,VARIACION,0,[11],"(862, 945)"
4,ACTIVOS,1,[20],"(33, 108)"


In [543]:

#print("numero de posiciones: " + str(len(coordenadas)))
#del_headEnd(df_data_organizer)
min_max_h,colums = extracion_colums(df_data_organizer)

#Dibujado de los rectangulos en la imagen
for p in min_max_h:
    cv2.rectangle(img, (p[0], hImg-5),(p[1], 5), (random.randint(0,255), random.randint(0,255), random.randint(0,255)), 1)

for p in coordenadas:
    cv2.rectangle(img, (p[0], p[1]),(p[2], p[3]), (50, 50, 255), 1)

cv2.imwrite('salida.png',img)

df = pd.DataFrame(build_df(df_data_organizer , colums))

df_copy = df.copy()




index, type_index = isNumericColum(df_copy)
#df.columns = index

df.to_excel("data.xlsx", index=False)




"""
img = cv2.resize(img, (600, 700))
cv2.imshow('Result', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
"""

el tamaño de cl es:  12
(385, 430) [0]
(526, 564) [1]
(703, 742) [2]
(862, 988) [3, 22, 28, 36, 40, 81, 10, 16, 32, 48, 54, 60, 65, 73, 77]
(33, 281) [4, 6, 42, 68, 38, 12, 24, 30, 33, 43, 44, 56, 62, 67, 74, 78, 5, 18, 50]
(33, 281) [5, 43, 4, 6, 42, 68, 38, 12, 24, 30, 33, 44, 56, 62, 67, 74, 78, 18, 50]
(402, 412) [7, 13, 19, 25, 45, 51, 57, 70]
(519, 625) [8, 14, 20, 26, 31, 34, 46, 52, 58, 63, 71, 75, 79]
(703, 808) [9, 15, 21, 27, 35, 39, 47, 53, 59, 64, 80]
(1012, 1051) [11, 17, 23, 29, 37, 41, 49, 55, 61, 66, 82]
(159, 350) [69]
(799, 808) [72, 76]
estos son a eliminar [[1], [2]]

[0, 7, 13, 19, 25, 45, 51, 57, 70]
[3, 22, 28, 36, 40, 81, 10, 16, 32, 48, 54, 60, 65, 73, 77]
[4, 6, 42, 68, 38, 12, 24, 30, 33, 43, 44, 56, 62, 67, 74, 78, 5, 18, 50, 5, 43, 4, 6, 42, 68, 38, 12, 24, 30, 33, 44, 56, 62, 67, 74, 78, 18, 50]
[8, 14, 20, 26, 31, 34, 46, 52, 58, 63, 71, 75, 79, 1]
[9, 15, 21, 27, 35, 39, 47, 53, 59, 64, 80, 2, 72, 76]
[11, 17, 23, 29, 37, 41, 49, 55, 61, 66, 82]
[69]
la

"\nimg = cv2.resize(img, (600, 700))\ncv2.imshow('Result', img)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n"

In [544]:
type_index

['text', 'num', 'num', 'num', 'num', 'text']

In [548]:
df.head(10)

,0,1,2,3,4,5
0,NaN,Notas,2021,2020,VARIACION,NaN
1,ACTIVOS,NaN,NaN,NaN,NaN,NaN
2,CORRIENTE,NaN,NaN,NaN,NaN,NaN
3,Efectivo,1,12068571,14864078,2795507,-19%
4,Instrumentos Financieros,2,153368544,147179446,6189098,4%
5,Cuentas comerciales por cobrar,3,225274730,86276995,138997735,161%
6,Deudores gubemamentales,4,32010000,14629464,17380536,119%
7,Deudores David Sandoval,NaN,1000000,NaN,1000000,NaN
8,Total Activo Corriente,NaN,423721845,262949983,160771862,61%
9,TOTAL ACTIVOS,NaN,NaN,262949983,160771862,61%
